In [1]:
#iniciar spark
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id

conf = SparkConf() 
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2')
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2,com.microsoft.azure:spark-mssql-connector_2.12:1.2.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.microsoft.azure#spark-mssql-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-05a4595b-0ef4-429d-9052-60a32bde0a84;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
	found com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 in central
	found com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 in central
:: resolution report :: resolve 503ms :: artifacts dl 21ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 from central in [default]
	com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	-------------------------------------------

In [2]:
dfCsv = spark.read.option('delimiter', ',').option('header', 'true').csv('s3a://ninastack-sptech-bucket-bruto/transacoes_pix.csv')

22/06/08 01:08:05 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [ ]:
dfCsv.show(vertical=True)

In [3]:
_dfCsv = dfCsv.select(
    col('valor').alias('VALOR TRANSFERIDO (R$)'),
    col('tipoChave').alias('TIPO CHAVE'),
    col('nomeInstituicao').alias('NOME INSTITUIÇÃO'),
    col('siglaEstado').alias('UF')
)
# _dfCsv.show()



In [4]:
dfCsvSingleCol = _dfCsv.select(concat(*_dfCsv.columns).alias('data'))
# dfCsvSingleCol.show()

In [ ]:
#dfCsvSingleCol.write.mode('append').text('s3a://ninastack-sptech-bucket-tratados/transacoes_pix.txt') 

In [5]:
dfTxt = spark.read.option('delimiter', ';').option('header', 'true').csv('s3a://ninastack-sptech-bucket-bruto/pessoa.txt')

In [ ]:
dfTxt.show()

In [6]:
_dfTxt = dfTxt.select(
    col('idade').cast(IntegerType()).alias('IDADE'),
)
_dfTxt.show()

+-----+
|IDADE|
+-----+
|   24|
|   44|
|   20|
|   45|
|   39|
|   36|
|   34|
|   37|
|   45|
|   33|
|   40|
|   18|
|   23|
|   30|
|   21|
|   18|
|   20|
|   32|
|   28|
|   36|
+-----+
only showing top 20 rows



In [7]:
dfTxtSingleCol = _dfTxt.select(concat(*_dfTxt.columns).alias('data'))
# dfTxtSingleCol.show()

In [ ]:
#dfTxtSingleCol.write.mode('append').text('s3a://ninastack-sptech-bucket-tratados/pessoa.txt')

In [8]:
dfJson = spark.read.json('s3a://ninastack-sptech-bucket-bruto/fraudes_pix.json')

In [ ]:
dfJson.show(vertical = True)

In [26]:
_dfJson = dfJson.select(
    col('valor').alias('VALOR DA TRANSAÇÃO')
)

TypeError: condition should be string or Column

In [10]:
dfJsonSingleCol = _dfJson.select(concat(*_dfJson.columns).alias('data'))
# dfJsonSingleCol.show()

In [ ]:
#dfJsonSingleCol.write.mode('append').text('s3a://ninastack-sptech-bucket-tratados/fraudes_pix.txt')

In [11]:
import pandas as pd
pdfXml = pd.read_xml('s3a://ninastack-sptech-bucket-bruto/DICT.xml')

In [12]:
dfXml = spark.createDataFrame(pdfXml)


In [13]:
_dfXml = dfXml.select(
    col('tipoConta').alias('TIPO CONTA'),
    col('reivindicadaDoacao').alias('CHAVE REIVINDICADA POR DOAÇÃO'),
)
# _dfXml.show()

In [14]:
dfXmlSingleCol = _dfXml.select(concat(*_dfXml.columns).alias('data'))


In [ ]:
#dfXmlSingleCol.write.mode('append').text('s3a://ninastack-sptech-bucket-tratados/DICT.txt')

In [15]:
_dfCsv = _dfCsv.withColumn("id", monotonically_increasing_id())

In [16]:
_dfTxt = _dfTxt.withColumn("id", monotonically_increasing_id())

In [17]:
_dfJson = _dfJson.withColumn("id", monotonically_increasing_id())

In [18]:
_dfXml = _dfXml.withColumn("id", monotonically_increasing_id())

In [19]:
df_seguranca = _dfCsv.join(_dfTxt, on="id", how="outer").join(_dfJson, on="id", how="outer").join(_dfXml, on="id", how="outer") 

In [ ]:
# df_sample = df_segurancaMaster.sample(0.001, seed=1234)

In [ ]:
# df_seguranca.write.option("header", True).csv("s3a://ninastack-sptech-bucket-cliente/security", emptyValue='') 

In [21]:
df_seguranca.write \
 .format('com.microsoft.sqlserver.jdbc.spark') \
 .mode('overwrite') \
 .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver') \
 .option('url', 'jdbc:sqlserver://server-gp-nina.database.windows.net;databaseName=database-gp06-nina;') \
 .option('dbtable', 'dbo.Security') \
 .option('user', 'adm') \
 .option('password', '@Urubu100') \
 .save()

22/06/08 01:13:08 WARN TaskSetManager: Stage 7 contains a task of very large size (5313 KiB). The maximum recommended task size is 1000 KiB.


In [22]:
import socket
print(socket.gethostbyname(socket.gethostname()))


10.100.0.245
